In [ ]:
%pip install torch transformers datasets tqdm
!mkdir checkpoints

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
%pip install datasets
!mkdir checkpoints

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [3]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader
import json


# Load necessary packages
from dataclasses import dataclass
from typing import List, Tuple

from torch.utils.data import Dataset
from transformers import PreTrainedTokenizerFast
from datasets import load_dataset

import torch.nn as nn
from torch.optim import Optimizer, AdamW
from tqdm.notebook import tqdm

In [35]:
import torch.nn.functional as F

In [ ]:
# from datasets import Dataset

In [ ]:
# from datasets import load_dataset

# ds = load_dataset("ChilleD/StrategyQA")

In [73]:
with open('./data/train.json', 'r') as file:
    train_data = json.load(file)

# Print the data
print(len(train_data))

2061


In [74]:
with open('./data/dev.json', 'r') as file:
    validation_data = json.load(file)

# Print the data
print(len(validation_data))

229


In [7]:
train_data[0]

{'qid': '2bc9c4f9c19c167187f2',
 'term': 'Genghis Khan',
 'description': 'founder and first Great Khan of the Mongol Empire',
 'question': 'Are more people today related to Genghis Khan than Julius Caesar?',
 'answer': True,
 'facts': ['Julius Caesar had three children.',
  'Genghis Khan had sixteen children.',
  'Modern geneticists have determined that  out of every 200 men today has DNA that can be traced to Genghis Khan.'],
 'decomposition': ['How many kids did Julius Caesar have?',
  'How many kids did Genghis Khan have?',
  'Is #2 greater than #1?'],
 'evidence': [[[['Caesarion-2', 'Julia (daughter of Caesar)-1']],
   [['Alakhai Bekhi-1', 'Tolui-1'], 'no_evidence'],
   ['operation']],
  [[['Julius Caesar-75']], [['Genghis Khan-17']], ['operation']],
  [[['Gaius Julius Caesar-7']],
   [['Genghis Khan-15'], 'no_evidence'],
   ['no_evidence', 'operation']]]}

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [39]:
import pickle

In [75]:
with open('./data/saved_variables.pkl', 'rb') as f:
    first_stage = pickle.load(f)
with open('./data/saved_variables_deepskeptic.pkl', 'rb') as f:
    second_stage = pickle.load(f)
with open('./data/saved_variables_falcskeptic.pkl', 'rb') as f:
    third_stage = pickle.load(f)

In [81]:
len(third_stage['generated_ans'])

229

In [97]:
for i in range(len(validation_data)):
  validation_data[i]['first_stage'] = first_stage['generated_ans'][i]
  validation_data[i]['second_stage'] = second_stage['generated_ans'][i]
  validation_data[i]['third_stage'] = third_stage['generated_ans'][i]

In [123]:
# Define the additional prompt
which_stage = 3
def make_prompt(data):
    """
    prompt = " Let's think step by step."
    # return data['question'] + prompt
    return data['question'] + ' ' + ' '.join(data['facts'])
    # return data['question']
    """

    prompt = data['question']
    num_facts = len(data['facts'])
    num_decomps = len(data['decomposition'])

    for i in range(max(num_facts, num_decomps)):
      if num_decomps > i:
        prompt += ' ' + data['decomposition'][i]
      if num_facts > i:
        prompt += ' ' + data['facts'][i]
    return prompt
def make_prompt2(data):
  prompt = data['question']
  if 'first_stage' not in data:
    return prompt
  if which_stage == 1:
    prompt += "\n" + data['first_stage']
  elif which_stage == 2:
    prompt += "\n1.\n" + data['second_stage']
  elif which_stage == 3:
    prompt += "\n" + data['third_stage']
  return prompt


In [99]:
@dataclass
class StratQAExample:
    """
    Convert a dict of raw data into an StratQAExample object that contains a text and label.
    """
    text: str
    label: int  # 0 for negative, 1 for positive

    @staticmethod
    def from_dict(data: dict):
        text = make_prompt2(data)
        if (data['answer'] == True):
            label = 1
        else:
            label = 0

        return StratQAExample(
            text=text,
            label=label,
        )


def initialize_datasets(tokenizer: PreTrainedTokenizerFast) -> dict:

    #with open('./data/train.json', 'r') as file:
        #train_data = json.load(file)
    #with open('./data/dev.json', 'r') as file:
        #validation_data = json.load(file)

    split_datasets = {}

    split_datasets['train'] = StratQADataset(tokenizer, train_data)
    split_datasets['validation'] = StratQADataset(tokenizer, validation_data)

    return split_datasets

In [100]:
class StratQADataset(Dataset):
    tokenizer: PreTrainedTokenizerFast = None

    def __init__(self, tokenizer: PreTrainedTokenizerFast, raw_data_list: List[dict]):
        StratQADataset.tokenizer = tokenizer
        self.sample_list = [StratQAExample.from_dict(data) for data in raw_data_list]

    def __len__(self):
        """
        Get the number of items in the dataset.
        """
        return len(self.sample_list)

    def __getitem__(self, idx):
        """
        Get the idx-th item from the dataset.
        """
        return self.sample_list[idx]

    def __iter__(self):
        """
        Get an iterator for the dataset.
        """
        return iter(self.sample_list)

    @staticmethod
    def collate_fn(batched_samples: List[StratQAExample]) -> dict:
        """
        Encode samples in batched_samples: tokenize the input texts, and turn labels into a tensor.
        :param batched_samples: A list of StratQAExample samples.
        :return: A dictionary of encoded texts and their corresponding labels (in tensors).
        """
        batched_text = [sample.text for sample in batched_samples]

        batched_label = [sample.label for sample in batched_samples]

        # Tokenize the input texts.
        text_encoding = StratQADataset.tokenizer(batched_text,
                                              padding=True,
                                              max_length=512,
                                              truncation=True,
                                              return_tensors="pt")

        label_encoding = torch.LongTensor(batched_label)

        return {
            'text_encoding': text_encoding,
            'label_encoding': label_encoding
        }

In [124]:
"""
Load train / validation / test dataset, using `initialize_datasets` in `dataset.py`.
"""
# load pre-trained tokenizer for roberta-base from transformers library.
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

datasets = initialize_datasets(tokenizer)

validation_first_element = datasets["validation"][0]

length_train = len(datasets["train"])
length_validation = len(datasets["validation"])

validation_dataloader = DataLoader(datasets['validation'],
                                   batch_size=64,
                                   shuffle=False,
                                   collate_fn=StratQADataset.collate_fn,
                                   num_workers=2)

batch = next(iter(validation_dataloader))

In [102]:
def train_one_epoch(model: nn.Module, dataloader: DataLoader, optimizer: Optimizer, epoch: int):
    model.train()

    with tqdm(dataloader, desc=f"Train Ep {epoch}", total=len(dataloader)) as tq:
        for batch in tq:
            text_encoding = batch["text_encoding"].to(model.device)
            label_encoding = batch['label_encoding'].to(model.device)

            loss = model(**text_encoding, labels=label_encoding).loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            tq.set_postfix({"loss": loss.detach().item()}) # for printing better-looking progress bar

In [55]:
all_probabilities1 = []
all_probabilities2 = []
all_probabilities3 = []

In [120]:
def evaluate(model: nn.Module, dataloader: DataLoader) -> float:
    model.eval()

    all_predictions = []
    all_labels = []

    all_probabilities = []

    with tqdm(dataloader, desc=f"Eval", total=len(dataloader)) as tq:
        for batch in tq:
            with torch.no_grad():
                text_encoding = batch["text_encoding"].to(model.device)
                label_encoding = batch['label_encoding'].to(model.device)

                logits = model(**text_encoding).logits

                probs = F.softmax(logits, dim=-1)

                predictions = logits.argmax(dim=-1).tolist()
                labels = label_encoding.tolist()
                probabilities = [tuple(p.tolist()) for p in probs]

                all_predictions += predictions
                all_labels += labels

                all_probabilities += probabilities
                #print(all_probabilities)

    # compute accuracy
    all_predictions = torch.Tensor(all_predictions)
    all_labels = torch.Tensor(all_labels)
    accuracy = compute_accuracy(all_predictions, all_labels)

    #print(all_probabilities)

    print(f"Accuracy: {accuracy}")
    return accuracy, all_probabilities[:]


def compute_accuracy(predictions: torch.Tensor, labels: torch.Tensor) -> float:
    assert predictions.size(-1) == labels.size(-1)

    accuracy = (predictions == labels).sum().item() / len(predictions)
    return accuracy

In [104]:
torch.manual_seed(64)

def main():
    # hyper-parameters 
    batch_size = 64
    learning_rate = 5e-5
    num_epochs = 10
    model_name = "roberta-base"

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    # load model on GPU.
    model = model.cuda()

    optimizer = AdamW(model.parameters(), lr=learning_rate, eps=1e-8)

    datasets = initialize_datasets(tokenizer)

    train_dataloader = DataLoader(datasets['train'],
                                   batch_size=batch_size,
                                   shuffle=True,
                                   collate_fn=StratQADataset.collate_fn,
                                   num_workers=2)
    validation_dataloader = DataLoader(datasets['validation'],
                                   batch_size=batch_size,
                                   shuffle=False,
                                   collate_fn=StratQADataset.collate_fn,
                                   num_workers=2)

    # training loop.
    best_acc = 0.0
    for epoch in range(1, num_epochs + 1):
        train_one_epoch(model, train_dataloader, optimizer, epoch)
        valid_acc, _ = evaluate(model, validation_dataloader)

        # saved checkpoint, save the new model in `./checkpoints` folder.
        if valid_acc > best_acc:
            best_acc = valid_acc

            # model_path = './checkpoints/model_{}.pth'.format(epoch)
            # torch.save(model.state_dict(), model_path)

            model_path = './checkpoints/best_model.pth'
            torch.save(model.state_dict(), model_path)

In [32]:
# Run the main training loop.
main()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Train Ep 1:   0%|          | 0/33 [00:00<?, ?it/s]

Eval:   0%|          | 0/4 [00:00<?, ?it/s]

Accuracy: 0.5327510917030568


Train Ep 2:   0%|          | 0/33 [00:00<?, ?it/s]

...

Eval:   0%|          | 0/4 [00:00<?, ?it/s]

Accuracy: 0.5327510917030568


Train Ep 10:   0%|          | 0/33 [00:00<?, ?it/s]

Eval:   0%|          | 0/4 [00:00<?, ?it/s]

Accuracy: 0.5327510917030568


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

model = AutoModelForSequenceClassification.from_pretrained("google/flan-t5-large")
model.load_state_dict(torch.load('./checkpoints/best_model.pth'))

datasets = initialize_datasets(tokenizer)
validation_dataloader = DataLoader(datasets['validation'],
                                   batch_size=64,
                                   shuffle=False,
                                   collate_fn=StratQADataset.collate_fn,
                                   num_workers=2)

valid_acc, _ = evaluate(model, validation_dataloader)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-82-76fa84c715b6>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowli

Eval:   0%|          | 0/4 [00:00<?, ?it/s]

Accuracy: 0.7248908296943232


In [125]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

model = AutoModelForSequenceClassification.from_pretrained("roberta-base")
model.load_state_dict(torch.load('./checkpoints/fact_finetuned.pth'))

datasets = initialize_datasets(tokenizer)
validation_dataloader = DataLoader(datasets['validation'],
                                   batch_size=64,
                                   shuffle=False,
                                   collate_fn=StratQADataset.collate_fn,
                                   num_workers=2)

if which_stage == 1:
  valid_acc, all_probabiltiies1 = evaluate(model, validation_dataloader)  # 0.5502183406113537
elif which_stage == 2:
  valid_acc, all_probabiltiies2 = evaluate(model, validation_dataloader) # 0.5458515283842795
elif which_stage == 3:
  valid_acc, all_probabiltiies3 = evaluate(model, validation_dataloader) # 0.5545851528384279


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Eval:   0%|          | 0/4 [00:00<?, ?it/s]

Accuracy: 0.5545851528384279


In [130]:
preds = []
for i in range(len(all_probabiltiies1)):
  num1 = max(all_probabiltiies1[i][0], all_probabiltiies1[i][1])
  num2 = max(all_probabiltiies2[i][0], all_probabiltiies2[i][1])
  num3 = max(all_probabiltiies3[i][0], all_probabiltiies3[i][1])
  pred = 0

  if num1 > num2 and num1 > num3:
    pred = 0 if all_probabiltiies1[i][0] > 0.5 else 1
  elif num2 > num1 and num2 > num3:
    pred = 0 if all_probabiltiies2[i][0] > 0.5 else 1
  else:
    pred = 0 if all_probabiltiies3[i][0] > 0.5 else 1
  preds.append(pred)

In [135]:
labels = [1 if d['answer'] else 0 for d in validation_data]

In [141]:
print("FINAL NUMBER")
print(sum([a==b for a,b in zip(preds, labels)]) / len(preds))

FINAL NUMBER
0.6157205240174672
